In [1]:
"""
#building the dictionary to use for the random ordering
1) Iterate through all the categories
2) Iterate through all the boundaries
3) Get the index and segment_ids for all and store them

#the way to make the final list
1) Loop through each size category
2) Loop through each type category
3) Select a random index from size of list
4) Filter the list to neurons that have not been picked
5) If list is 0 then reset the list and filter the list again
6) Pick a random index from the list
    a. Add that segment to the list
    b. Add the neuron as already being picked
7) Repeat until the picked list is of a N length

8) Make sure at the end have something from every neuron by finding unique segments
"""



'\n#building the dictionary to use for the random ordering\n1) Iterate through all the categories\n2) Iterate through all the boundaries\n3) Get the index and segment_ids for all and store them\n\n#the way to make the final list\n1) Loop through each size category\n2) Loop through each type category\n3) Select a random index from size of list\n4) Filter the list to neurons that have not been picked\n5) If list is 0 then reset the list and filter the list again\n6) Pick a random index from the list\n    a. Add that segment to the list\n    b. Add the neuron as already being picked\n7) Repeat until the picked list is of a N length\n\n8) Make sure at the end have something from every neuron by finding unique segments\n'

In [2]:
import datajoint as dj
import numpy as np
import time
import seaborn as sns
import random
sns.set()

In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema("microns_pinky")
pinky = dj.create_virtual_module("pinky","microns_pinky")

Connecting celiib@10.28.0.34:3306


In [12]:
total_compartments = ["Dendrite"] 

compartment_size_boundaries = dict([(k,[]) for k in total_compartments])
#calculate the size groups for each of the compartments
for comp_type in compartment_size_boundaries.keys():
    search_key = dict(segmentation=3,decimation_ratio=0.35,compartment_type = comp_type)
    excitatory_n_vertices = (pinky.CompartmentOrphan.ComponentOrphan() & search_key).fetch("n_vertex_indices")
    excitatory_n_vertices_size_filtered = excitatory_n_vertices[excitatory_n_vertices>5000]

    #get the quartiles for that group
    
    percentiles_10_through_90 = [np.percentile(excitatory_n_vertices_size_filtered, k*10) for k in range(1,10) ]
    print("percentiles_10_through_90 = " + str(percentiles_10_through_90))
    iq_average = np.mean([percentiles_10_through_90[i+1] - percentiles_10_through_90[i] for i in range(0,len(percentiles_10_through_90)-1)])
    print("iq_average = " + str(iq_average))
    
    lower_bounds = percentiles_10_through_90[0] - iq_average
    upper_bounds = percentiles_10_through_90[8] + iq_average


    #check that these numbers aren't too low or too high
    if lower_bounds < np.min(excitatory_n_vertices_size_filtered):
        print(f"lower bounds value of {lower_bounds} was too low so changing to the minimum value")
        lower_bounds = np.min(excitatory_n_vertices_size_filtered)
        
    print("lower_bounds = " + str(lower_bounds))

    if upper_bounds > np.max(excitatory_n_vertices_size_filtered):
        print(f"upper bounds value of {upper_bounds} was too low so changing to the minimum value")
        upper_bounds = np.max(excitatory_n_vertices_size_filtered)
        
    print("upper_bounds = " + str(upper_bounds))
        
    boundary_marks = [lower_bounds] + percentiles_10_through_90 + [upper_bounds]
    
    
    compartment_size_boundaries[comp_type] = [(boundary_marks[i],boundary_marks[i+1])
                                                      for i in range(0,len(boundary_marks)-1)]
    
    print("compartment_size_boundaries = " + str(compartment_size_boundaries))
    
    
    
    
    #print("boundaries = " + str(boundaries))
    

percentiles_10_through_90 = [8562.2, 11640.0, 15160.2, 18786.600000000002, 24985.0, 32695.800000000003, 42914.8, 57227.20000000001, 82560.20000000001]
iq_average = 9249.750000000002
lower bounds value of -687.5500000000011 was too low so changing to the minimum value
lower_bounds = 5035
upper_bounds = 91809.95000000001
compartment_size_boundaries = {'Dendrite': [(5035, 8562.2), (8562.2, 11640.0), (11640.0, 15160.2), (15160.2, 18786.600000000002), (18786.600000000002, 24985.0), (24985.0, 32695.800000000003), (32695.800000000003, 42914.8), (42914.8, 57227.20000000001), (57227.20000000001, 82560.20000000001), (82560.20000000001, 91809.95000000001)]}


5035

In [13]:
compartment_size_boundaries

{'Dendrite': [(5035, 8562.2),
  (8562.2, 11640.0),
  (11640.0, 15160.2),
  (15160.2, 18786.600000000002),
  (18786.600000000002, 24985.0),
  (24985.0, 32695.800000000003),
  (32695.800000000003, 42914.8),
  (42914.8, 57227.20000000001),
  (57227.20000000001, 82560.20000000001),
  (82560.20000000001, 91809.95000000001)]}

In [19]:
#go through and assemble the dictionary pull down the lists
total_sizes = ["group" + str(i) for i in range(1,11)]
compartment_size_lists = dict([(k,dict([(j,[]) for j in total_sizes]))
                              for k in total_compartments])

#interate through and build lists:
for y,size_group in enumerate(total_sizes):
    for comp_type in compartment_size_lists.keys():
    
        print(f"Working on comp {comp_type} {size_group}")
        #get the boundaries
        boundaries = compartment_size_boundaries[comp_type][y]
        print(f"Boundaries = {boundaries}")
        
        boundary_restrictions = "n_vertex_indices >= " + str(boundaries[0]) + " AND n_vertex_indices <= " + str(boundaries[1])
        print("boundary_restrictions = " + str(boundary_restrictions))

        search_key = dict(segmentation=3,decimation_ratio=0.35,compartment_type = comp_type)
        excitatory_n_vertices_boundary = pinky.CompartmentOrphan.ComponentOrphan() & search_key & boundary_restrictions#).fetch("n_vertex_indices")
        n_vertex_indices_with_boundaries = excitatory_n_vertices_boundary.fetch("n_vertex_indices")
        print("Min and Max = " + str((np.min(n_vertex_indices_with_boundaries),np.max(n_vertex_indices_with_boundaries))))

        """
        At this point have all the components within a size range of that specific compartment type but do not have them grouped by neurons
        """
        seg_ids,comp_index = excitatory_n_vertices_boundary.fetch("segment_id","component_index")
        total_pairs = np.stack([comp_index,seg_ids],axis=1)
        print("Total Number of pairs = " + str(total_pairs.shape[0]))
        print(total_pairs)
        compartment_size_lists[comp_type][size_group] = total_pairs



Working on comp Dendrite group1
Boundaries = (5035, 8562.2)
boundary_restrictions = n_vertex_indices >= 5035 AND n_vertex_indices <= 8562.2


/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:150: UserWarning: Mysql server has gone away. Reconnecting to the server.
  warnings.warn("Mysql server has gone away. Reconnecting to the server.")


Min and Max = (5035, 8539)
Total Number of pairs = 63
[[                 0 648518346341361112]
 [                 0 648518346341381313]
 [                 0 648518346341382814]
 [                 0 648518346341384400]
 [                 0 648518346341400619]
 [                 0 648518346341401158]
 [                 0 648518346341402769]
 [                 0 648518346341416638]
 [                 0 648518346341787511]
 [                 0 648518346341888873]
 [                 0 648518346342157256]
 [                 0 648518346342406746]
 [                 0 648518346342412201]
 [                 0 648518346342415678]
 [                 0 648518346342463843]
 [                 0 648518346342467480]
 [                 0 648518346342788005]
 [                 0 648518346342802094]
 [                 0 648518346342805766]
 [                 0 648518346342920209]
 [                 0 648518346343164555]
 [                 0 648518346343624053]
 [                 0 648518346343624383]
 [ 

In [15]:
import copy 
compartment_size_lists_saved = copy.deepcopy(compartment_size_lists)

In [16]:
compartment_size_lists = copy.deepcopy(compartment_size_lists_saved)

In [17]:
segmentation = 3
decimation_ratio = 0.35

already_used = []
total_list = []
length_threshold = 200
counter = 0
while(len(total_list) < length_threshold):
    for size in total_sizes:
        for comp_type in total_compartments:
            repeat = True
            while repeat == True:
                
                #print(f"Working on {comp_type} {size}")
                #get the current list of neurons
                current_array = compartment_size_lists[comp_type][size][:,1]


                indices = np.where(np.invert(np.in1d(current_array,np.array(already_used))))[0]
    #             #print("indices = " + str(indices))
    #             #print("len(indices) = " + str(len(indices)))
    #             current_filtered_array = current_array[indices]

    #             #print(current_filtered_array)
    #             #check if any in the already_used:
    #             current_filtered_array_list = list(current_filtered_array)
    #             #print("current_filtered_array_list = " + str(current_filtered_array_list))
    #             #print("already_used = " + str(already_used))
    #             print("intersection = " + str(set(current_filtered_array_list).intersection(set(already_used))))
    #             for i in current_filtered_array_list:
    #                 if i in already_used:
    #                     raise Exception("Not allowed Neuron")



                if len(indices) <= 0:
                    print("Resetting the already_used list when it's current length is " + str(len(already_used)))
                    already_used = []
                    indices = np.where(np.invert(np.in1d(current_array,np.array(already_used))))[0]

                #get the random int 
                random_index = random.randint(0,len(indices) - 1)
                neuron_index = indices[random_index]

                #pick the neuron
                picked_neuron = compartment_size_lists[comp_type][size][neuron_index]

                final_dict = dict(process_order = counter,
                                      segmentation=segmentation,
                                      segment_id=picked_neuron[1],
                                      decimation_ratio=decimation_ratio,
                                      compartment_type=comp_type,
                                      component_index=picked_neuron[0])
                
                #check that this hasn't already been added to the list
                repeat = False
                for t in total_list:
                    if t == final_dict:
                        print("Found a repeat")
                        repeat = True
                        break

            
            total_list.append(final_dict)
            counter += 1
            #delete the neruon from the list
            compartment_size_lists[comp_type][size] = np.delete(compartment_size_lists[comp_type][size],random_index,axis=0)

            #add the segment_id to those already visited
            already_used.append(picked_neuron[1])
            
            #print("Length of already_used= " + str(len(np.unique(already_used))))
        

Resetting the already_used list when it's current length is 179


ValueError: empty range for randrange() (0,0, 0)

In [20]:
#check the list has values in it
total_list

[{'process_order': 0,
  'segmentation': 3,
  'segment_id': 648518346341382814,
  'decimation_ratio': 0.35,
  'compartment_type': 'Dendrite',
  'component_index': 0},
 {'process_order': 1,
  'segmentation': 3,
  'segment_id': 648518346342157584,
  'decimation_ratio': 0.35,
  'compartment_type': 'Dendrite',
  'component_index': 0},
 {'process_order': 2,
  'segmentation': 3,
  'segment_id': 648518346348719136,
  'decimation_ratio': 0.35,
  'compartment_type': 'Dendrite',
  'component_index': 0},
 {'process_order': 3,
  'segmentation': 3,
  'segment_id': 648518346341403533,
  'decimation_ratio': 0.35,
  'compartment_type': 'Dendrite',
  'component_index': 0},
 {'process_order': 4,
  'segmentation': 3,
  'segment_id': 648518346344517719,
  'decimation_ratio': 0.35,
  'compartment_type': 'Dendrite',
  'component_index': 0},
 {'process_order': 5,
  'segmentation': 3,
  'segment_id': 648518346341392729,
  'decimation_ratio': 0.35,
  'compartment_type': 'Dendrite',
  'component_index': 0},
 {'p

# HAD TO MANUALLY STOP THE ONE ABOVE

In [113]:
len(total_list)

408

In [ ]:
"""
segmentation         : tinyint unsigned             # segmentation id
    segment_id           : bigint unsigned              # segment id unique within each Segmentation
    decimation_ratio     : decimal(3,2) 
    compartment_type     : varchar(16)                  # Basal, Apical, spine head, etc.
    component_index      : smallint unsigned            # Which sub-compartment of a certain label this is.
"""

In [23]:
SpineValidationCompartmentOrphanComponentOrphan.drop()

`microns_pinky`.`spine_validation_compartment_orphan_component_orphan` (0 tuples)


Proceed? [yes, No]:  yes


Tables dropped.  Restart kernel.


In [24]:
# create a new datajoint table to store the values
schema = dj.schema("microns_pinky")

@schema
class SpineValidationCompartmentOrphanComponentOrphan(dj.Manual):
    definition="""
    process_order        : bigint unsigned              # segment id unique within each Segmentation
    -> pinky.CompartmentOrphan.ComponentOrphan
    ---
    
    """

In [26]:
SpineValidationCompartmentOrphanComponentOrphan.insert(total_list)

In [ ]:
#might have to save this file as a list

In [27]:
spine_validation_table = (SpineValidationCompartmentOrphanComponentOrphan() * pinky.CompartmentOrphan.ComponentOrphan).proj("n_vertex_indices","n_triangle_indices")




In [28]:
whole_data = spine_validation_table.fetch(order_by="process_order")

In [29]:
#demonstrates that have effectively created an ordered list 
import pandas as pd
df = pd.DataFrame(whole_data)
df

,process_order,segmentation,segment_id,decimation_ratio,compartment_type,component_index,n_vertex_indices,n_triangle_indices
0,0,3,648518346341382814,0.35,Dendrite,0,7558,15105
1,1,3,648518346342157584,0.35,Dendrite,0,9921,19794
2,2,3,648518346348719136,0.35,Dendrite,0,12335,24605
3,3,3,648518346341403533,0.35,Dendrite,0,16037,32011
4,4,3,648518346344517719,0.35,Dendrite,0,21579,42960
5,5,3,648518346341392729,0.35,Dendrite,0,27236,54379
6,6,3,648518346342796769,0.35,Dendrite,0,34190,68121
7,7,3,648518346341414523,0.35,Dendrite,0,53758,107274
8,8,3,648518346342803480,0.35,Dendrite,0,82507,164714
9,9,3,648518346341360739,0.35,Dendrite,0,90632,180808
